In [1]:

import tensorflow as tf
print(tf.__version__)


2.16.1


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\Project with Dr. Mumbi\LB_CC_4.4"
label_dir = r"C:\Users\net pc\Desktop\Target\LB_CC_4.4"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:
# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')
   
    return model


In [10]:
def define_generator(image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return [X1, X2], y


In [12]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [13]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    
    # Input tensor for the generator model
    gen_input = Input(shape=image_shape)
    
    # Connect the generator output to the discriminator input
    gan_output = d_model([gen_input, g_model(gen_input)])
    
    # Define GAN model taking generator input and outputting discriminator decision
    gan_model = Model(gen_input, gan_output)
    
    # Compile GAN model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy'], optimizer=opt)
    
    return gan_model

# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

C:\Users\Public\Efendim\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [14]:

import numpy as np

def train(d_model, g_model, gan_model, dataset, n_epochs=10000, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # Unpack dataset
    trainA, trainB = dataset
    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # Calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # Select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # Generate a batch of fake samples
        X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_patch)
        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], np.ones((n_batch, n_patch, n_patch, 1)))
        # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], np.zeros((n_batch, n_patch, n_patch, 1)))
        # Update the generator
        g_loss = gan_model.train_on_batch(X_realA, [np.ones((n_batch, n_patch, n_patch, 1)), X_realB])
        # Summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # Summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 
import os
from PIL import Image

from PIL import Image

def save_img(filename, image):
    """Save the image to the specified filename."""
    im = Image.fromarray((image * 255).astype(np.uint8))
    im.save(filename)

# Summarize performance
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # Select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # Generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # Scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0

    output_path = "C:\\Users\\net pc\\Desktop\\New folder\\LB_CC_4.4"

    for i in range(n_samples):
        # Save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # Save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # Save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # Save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")


In [16]:


# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=1000, n_batch=1)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step


C:\Users\Public\Efendim\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1[0.692] d2[0.693] g[0.693]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>2, d1[0.693] d2[0.693] g[0.680]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>3, d1[0.693] d2[0.693] g[0.667]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>4, d1[0.693] d2[0.693] g[0.653]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>5, d1[0.693] d2[0.693] g[0.639]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>6, d1[0.693] d2[0.693] g[0.626]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>7, d1[0.693] d2[0.693] g[0.615]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>8, d1[0.693] d2[0.693] g[0.604]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>9, d1[0.693] d2[0.693] g[0.595]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>10, d1[0.693] d2[0.693] g[0.586]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step


Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>11, d1[0.693] d2[0.693] g[0.578]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>12, d1[0.693] d2[0.694] g[0.571]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>13, d1[0.694] d2[0.694] g[0.564]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>14, d1[0.694] d2[0.694] g[0.558]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>15, d1[0.694] d2[0.694] g[0.552]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
>16, d1[0.694] d2[0.694] g[0.547]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>17, d1[0.694] d2[0.694] g[0.542]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>18, d1[0.694] d2[0.695] g[0.538]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
>19, d1[0.694] d2[0.695] g[0.533]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>20, d1[0.695] d2[0.695] g[0.529]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>21, d1[0.695] d2[0.695] g[0.526]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>22, d1[0.695] d2[0.695] g[0.522]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>23, d1[0.695] d2[0.696] g[0.519]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>24, d1[0.695] d2[0.696] g[0.516]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
>25, d1[0.696] d2[0.696] g[0.513]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>26, d1[0.696] d2[0.696] g[0.510]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>27, d1[0.696] d2[0.696] g[0.507]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>28, d1[0.696] d2[0.697] g[0.504]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>29, d1[0.697] d2[0.697] g[0.502]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>30, d1[0.697] d2[0.697] g[0.500]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


...

Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>51, d1[0.703] d2[0.703] g[0.465]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>52, d1[0.703] d2[0.703] g[0.464]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
>53, d1[0.703] d2[0.704] g[0.463]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>54, d1[0.703] d2[0.704] g[0.462]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>55, d1[0.704] d2[0.704] g[0.461]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>56, d1[0.704] d2[0.705] g[0.460]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
>57, d1[0.704] d2[0.705] g[0.458]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>58, d1[0.705] d2[0.705] g[0.457]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>59, d1[0.705] d2[0.705] g[0.456]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>60, d1[0.705] d2[0.706] g[0.455]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>61, d1[0.706] d2[0.706] g[0.454]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>62, d1[0.706] d2[0.706] g[0.453]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
>63, d1[0.706] d2[0.707] g[0.452]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>64, d1[0.707] d2[0.707] g[0.452]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>65, d1[0.707] d2[0.707] g[0.451]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>66, d1[0.707] d2[0.708] g[0.450]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>67, d1[0.708] d2[0.708] g[0.449]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>68, d1[0.708] d2[0.708] g[0.448]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>69, d1[0.708] d2[0.709] g[0.447]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>70, d1[0.709] d2[0.709] g[0.446]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>71, d1[0.709] d2[0.709] g[0.445]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>72, d1[0.709] d2[0.710] g[0.445]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>73, d1[0.710] d2[0.710] g[0.444]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>74, d1[0.710] d2[0.710] g[0.443]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>75, d1[0.710] d2[0.711] g[0.442]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>76, d1[0.711] d2[0.711] g[0.442]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>77, d1[0.711] d2[0.712] g[0.441]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>78, d1[0.711] d2[0.712] g[0.440]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>79, d1[0.712] d2[0.712] g[0.439]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>80, d1[0.712] d2[0.713] g[0.439]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


Images and model saved at C:\Users\net pc\Desktop\New folder\LB_CC_4.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>81, d1[0.712] d2[0.713] g[0.438]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>82, d1[0.713] d2[0.713] g[0.437]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>83, d1[0.713] d2[0.714] g[0.437]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>84, d1[0.714] d2[0.714] g[0.436]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>85, d1[0.714] d2[0.714] g[0.435]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>86, d1[0.714] d2[0.715] g[0.434]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>87, d1[0.715] d2[0.715] g[0.434]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>88, d1[0.715] d2[0.715] g[0.433]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>89, d1[0.715] d2[0.716] g[0.433]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>90, d1[0.716] d2[0.716] g[0.432]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\net pc\\Desktop\\New folder\\LB_CC_4.4\\real_source_89_1.jpeg'